# CoC to Census Tract Mapping
In this section, we will associate each Continuum of Care (CoC) with relevant census data, such as census level demographics, income, and cost of living data. To do so, we will use shape files of each CoC and US Census Bureau tracts. Note that tracts may change on an annual basis; therefore, these data must be generated for each year.

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import numpy as np
from shapely.geometry import Point
from urllib.request import urlretrieve
from urllib.parse import urlparse
from utils import *
from parameters import *

In [24]:
# Run this cell to clear variables
%reset

## Algorithm
CoCs can contain many US Census Bureau tracts. For each tract constituting a CoC, we note the fraction of the total tract encompassed in the CoC. Tracts of which less than 1% of the total tract area is within the CoC are omitted.

Algorithmically, this is done by performing intersections between shape files and then calculating the total area of overlap after converting to an area-preserving coordinate reference system. The data are stored in dictionaries for each state.

In [2]:
def create_cocs_tract_crosswalk(state = 'MA', tracts_dir = 'tl_2024_25_tract', year = 2024):
    
    directory = Path.cwd()
    coc_path = directory / 'data' / 'coc-shapefiles' / str(year) / f'CoC_GIS_State_Shapefile_{state}' / state_info[state][0]
    tracts_path = directory / 'data' / tracts_dir
    cocs = [coc for coc in os.listdir(coc_path) if coc.startswith(state + '_')]

    # We must represent each CoC as a combination of Census tracts
    cocs_tract_crosswalk = {}
    tracts = gpd.read_file(tracts_path / str(tracts_dir + '.shp'))

    for coc in cocs:
        coc_tract_crosswalk = {}
        coc_file = coc_path / coc / str(coc + '.shp')
        if coc_file.is_file():
            coc_gpd = gpd.read_file(coc_file)
            overlapping_tracts = gpd.sjoin(tracts, coc_gpd, how="inner", predicate="intersects")
            # FOR DEBUGGING: Plot the overlapping tracts
            # fig, axes = plt.subplots(ncols = 2)
            # coc_gpd.plot(
            #     ax=axes[0]
            # )
            # overlapping_tracts.plot(
            #     ax=axes[1]
            # )
            
            # For those tracts that overlap, we estimate how much of each tract is contained in the CoC
            # First, project the CoC and tracts into an area-preserving coordinate system
            coc_projected = coc_gpd.to_crs(epsg=6933)
            tracts_projected = overlapping_tracts.to_crs(epsg=6933)
            for tract in tracts_projected.itertuples():
                intersection_area = tract.geometry.intersection(coc_projected.geometry).area
                tract_area = tract.geometry.area
                overlap = (intersection_area.iat[0]/tract_area)
            
                # Only keep tracts for which at least 1% of the tract is in CoC
                if overlap > 0.01: coc_tract_crosswalk[tract.GEOID] = round(overlap, 4)
        
        cocs_tract_crosswalk[coc] = coc_tract_crosswalk
    
    return cocs_tract_crosswalk

ma_cocs_tract_crosswalk = create_cocs_tract_crosswalk(state='MA', tracts_dir='tl_2024_25_tract', year=2024)

In [74]:

years = range(2007, 2024)

# for year in range(2007, 2025):
#     for state in states:
#         download_coc_state_file(state, year)
# for year in range(2007, 2025):
#     unzip_all(f'data/coc-shapefiles/{year}')




'Alaska'

In [2]:
download_tract_file("MA", 2024)

Downloading: https://www2.census.gov/geo/tiger/TIGER2024/TRACT/tl_2024_25_tract.zip
Saving to: data/tract-shapefiles/2024/Tract_GIS_ShapeFile_MA.zip
--------------------------------------------------
Progress: 1.5% (0.1/4.3 MB) - Speed: 4.1 MB/ss

/Users/nauman/.pyenv/versions/3.13.0/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www2.census.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Progress: 100.0% (4.3/4.3 MB) - Speed: 32.2 MB/s
✅ Download complete!
File size: 4.3 MB
Time taken: 0.1 seconds
Average speed: 32.2 MB/s
Saved to: /Users/nauman/Documents/mit/summer-2025/uplift/15-uplift/homelessness-prediction/data/tract-shapefiles/2024/Tract_GIS_ShapeFile_MA.zip


'data/tract-shapefiles/2024/Tract_GIS_ShapeFile_MA.zip'